In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys, os, glob, subprocess, random, time
from IPython.display import display, HTML
mpl.rc('xtick', labelsize=13) 
mpl.rc('ytick', labelsize=13) 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import networkx as nx
try:
    print(open('README.md').read())
except:
    pass
print('-'*100)
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        if 'git' in str(root):
            continue
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
list_files('.')

def prints(txt):
    txt = str(txt)
    try:
        print(txt[:1000])
    except:
        pass
    #print(txt)


# LumPy
[![License: GPL v3](https://img.shields.io/badge/License-GPL%20v3-blue.svg)](http://www.gnu.org/licenses/gpl-3.0)
[![Build Status](https://travis-ci.org/gerritgr/LumPyQest.svg?branch=master)](https://travis-ci.org/gerritgr/LumPyQest)

Copyright: 2018, Gerrit Großmann, [Group of Modeling and Simulation](https://mosi.uni-saarland.de/) at [Saarland University](http://www.cs.uni-saarland.de/)

Version: 0.1 (Please note that this code is an experimental version in a very early development stage.)
## Overview
------------------
The LumPy toolset provides a proof of concept for lumping for AME equations for multistate processes on complex networks.
It reduces the large number of ODEs given by the equation systems by clustering them and only solving a single ODE per cluster.
LumPy is written in Python 3 (requiring SciPy) and published under GPL v3 license.

As input, the tool takes model descriptions (containing degree distribution, rules,
time horizon, etc.) and outputs the lumped (o

In [2]:
clustering_file = 'output/SIS/clustering_SIS_306.csv'
clustering_original_file = 'output/SISfull/clustering_SISfull_1326.csv'
dynamics_original_file = 'output/SISfull/ame_SISfull_1326_trajectories_individual.csv'
dynamics_reduced_file = 'output/SIS/ame_SIS_306_trajectories_individual.csv'  #73

clustering_file = 'output/SIS/clustering_SIS_73.csv'
dynamics_reduced_file = 'output/SIS/ame_SIS_73_trajectories_individual.csv'  #73

clustering = pd.read_csv(clustering_file, skiprows=1, sep=';') 
clustering_original = pd.read_csv(clustering_original_file, skiprows=1, sep=';') 
dynamics_original = pd.read_csv(dynamics_original_file, skiprows=1, sep=';')  #open(dynamics_original_file).read()
dynamics_reduced = pd.read_csv(dynamics_reduced_file, skiprows=1, sep=';')  #open(dynamics_reduced_file).read()



In [3]:
clustering_original.head()
len(set(list(clustering['cluster'])))

73

# Funtion m to cluster and vice versa

In [4]:
def get_cluster_maps(clustering):
    try:
        clustering = clustering.to_dict(orient='records')
    except: 
        pass
    
    m_to_cluster = dict()
    cluster_to_m = dict()
    m_to_weight = dict()
    for line in clustering:
        m = eval(line['m'])
        cluster = line['cluster']
        weight = line['weight']
        m_to_cluster[m] = cluster
        m_to_weight[m] = weight
        if cluster not in cluster_to_m:
            cluster_to_m[cluster] = list()
        cluster_to_m[cluster].append(m)
    return m_to_cluster, cluster_to_m, m_to_weight

# Plot Original

### 2652 ODEs and 1326 cluster

In [5]:
print(clustering_original.head())
m_to_cluster, cluster_to_m, m_to_weight = get_cluster_maps(clustering_original)
print(str(cluster_to_m)[:1000])

          m               cluster  weight
0    (7, 3)  (10, (699.0, 300.0))     1.0
1   (31, 6)  (37, (837.0, 162.0))     1.0
2   (5, 31)  (36, (138.0, 861.0))     1.0
3  (20, 25)  (45, (444.0, 555.0))     1.0
4   (3, 35)   (38, (78.0, 921.0))     1.0
{'(10, (699.0, 300.0))': [(7, 3)], '(37, (837.0, 162.0))': [(31, 6)], '(36, (138.0, 861.0))': [(5, 31)], '(45, (444.0, 555.0))': [(20, 25)], '(38, (78.0, 921.0))': [(3, 35)], '(25, (639.0, 360.0))': [(16, 9)], '(34, (176.0, 823.0))': [(6, 28)], '(49, (428.0, 571.0))': [(21, 28)], '(40, (99.0, 900.0))': [(4, 36)], '(23, (826.0, 173.0))': [(19, 4)], '(46, (934.0, 65.0))': [(43, 3)], '(37, (459.0, 540.0))': [(17, 20)], '(32, (218.0, 781.0))': [(7, 25)], '(41, (536.0, 463.0))': [(22, 19)], '(27, (740.0, 259.0))': [(20, 7)], '(48, (916.0, 83.0))': [(44, 4)], '(37, (486.0, 513.0))': [(18, 19)], '(49, (469.0, 530.0))': [(23, 26)], '(27, (777.0, 222.0))': [(21, 6)], '(13, (615.0, 384.0))': [(8, 5)], '(34, (941.0, 58.0))': [(32, 2)], '(45, (288.0,

In [6]:
dynamics_original.head()

,"('I', '(0, (0.0, 999.0))')","('I', '(1, (0.0, 999.0))')","('I', '(1, (999.0, 0.0))')","('I', '(10, (0.0, 999.0))')","('I', '(10, (199.0, 800.0))')","('I', '(10, (299.0, 700.0))')","('I', '(10, (399.0, 600.0))')","('I', '(10, (499.0, 500.0))')","('I', '(10, (599.0, 400.0))')","('I', '(10, (699.0, 300.0))')",...,"('S', '(9, (0.0, 999.0))')","('S', '(9, (111.0, 888.0))')","('S', '(9, (222.0, 777.0))')","('S', '(9, (333.0, 666.0))')","('S', '(9, (444.0, 555.0))')","('S', '(9, (555.0, 444.0))')","('S', '(9, (666.0, 333.0))')","('S', '(9, (777.0, 222.0))')","('S', '(9, (888.0, 111.0))')","('S', '(9, (999.0, 0.0))')"
0,7.464749e-11,0.067183,0.007465,8.230767e-05,0.000046,0.000014,0.000003,3.512597e-07,3.252405e-08,2.065019e-09,...,0.001071,0.001071,0.000476,0.000123,0.000021,0.000002,1.693005e-07,8.061929e-09,2.239425e-10,2.764722e-12
1,7.136282e-11,0.052267,0.030967,1.030474e-05,0.000160,0.000160,0.000100,4.285951e-05,1.274047e-05,2.611262e-06,...,0.000600,0.000889,0.000586,0.000227,0.000057,0.000010,1.097069e-06,8.416672e-08,4.013849e-09,9.346394e-11
2,6.822267e-11,0.041662,0.053330,2.126014e-06,0.000150,0.000227,0.000224,1.565866e-04,7.852179e-05,2.776193e-05,...,0.000340,0.000650,0.000555,0.000280,0.000093,0.000022,3.843804e-06,5.232218e-07,5.188697e-08,2.749108e-09
3,6.522070e-11,0.034487,0.074257,9.199038e-07,0.000120,0.000223,0.000284,2.652625e-04,1.841542e-04,9.230811e-05,...,0.000207,0.000460,0.000459,0.000273,0.000110,0.000033,8.451928e-06,1.857919e-06,3.054601e-07,2.578077e-08
4,6.235083e-11,0.029845,0.093532,5.771082e-07,0.000091,0.000193,0.000287,3.275440e-04,2.859434e-04,1.843058e-04,...,0.000136,0.000328,0.000358,0.000236,0.000108,0.000040,1.383653e-05,4.349116e-06,9.995462e-07,1.144244e-07


In [8]:
os.system('mkdir gif_out')

def plot_dyn(m_to_value, title='Original I Fraction', t=0.3, clusters=100):
    import matplotlib.cm as cm
    x_list = list()
    y_list = list()
    value_list = list()
    clusters = len(set(m_to_value.values())) #does not work, need to use clusters
    
    for m, value in m_to_value.items():
        x_list.append(m[0])
        y_list.append(m[1])
        value_list.append(value)
        
        
    value_list = [np.log(n+0.0000000001) for n in value_list]    
    #plt.scatter(x_list, y_list, c=value_list, cmap='hot')
    #plt.title(title)
    #plt.show()
    #plt.savefig('gif_out/'+title.replace(' ','')+'.jpg')
    
    
    ax = plt.gca()
    ax.scatter(x_list, y_list, c=value_list, alpha=0.9, linewidths=0.0, cmap=cm.plasma, marker=',', s=16)
    ax.set_xlim([-0.5, 55])
    ax.set_ylim([-0.5, 55])
    ax.set_aspect(1.0)
    ax.set(xlabel='Neighbors in state I', ylabel='Neighbors in state S')
    t = str(t)
    t += '000000000'
    t = t[:5]
    plt.title('Probability Mass at t={}, {} clusters'.format(t,clusters))
    plt.savefig('gif_out/'+title.replace(' ','')+'.jpg' , dpi=1000)
    

In [ ]:
m_to_value_t = dict()
for timepoint in list(range(101)):
    dynamics_original   #101 rows × 2652 columns
    dyn_o =  dynamics_original.to_dict(orient='records')
    dyn_line = dyn_o[timepoint]
    m_to_value = dict()
    for cluster, value in dyn_line.items():
        if 'S' in cluster:
            continue
        cluster = cluster.replace("('I', '", "").replace("')","")
        mlist = cluster_to_m[cluster]
        for m in mlist:   # do weights
            m_to_value[m] = value
    prints(m_to_value) 
    m_to_value_t[timepoint] = dict(m_to_value)

    real_time = str(timepoint/101.0*3) + '00000000000000000000'
    real_time = real_time[:6]
    plot_dyn(m_to_value, 'Original Inf   t='+real_time, timepoint/101.0*3)

{(0, 0): 7.464749463148459e-11, (0, 1): 0.06718274516833614, (1, 0): 0.007464749463148461, (0, 10): 8.230767434916922e-05, (2, 8): 4.572648574953849e-05, (3, 7): 1.3548588370233603e-05, (4, 6): 2.634447738656541e-06, (5, 5): 3.5125969848753935e-07, (6, 4): 3.252404615625367e-08, (7, 3): 2.065018803571661e-09, (8, 2): 8.604245014881892e-11, (9, 1): 2.124504941946136e-12, (1, 9): 9.14529714990768e-05, (10, 0): 2.3605610466068345e-14, (0, 11): 5.837153675187414e-05, (2, 9): 3.963499409077874e-05, (3, 8): 1.3211664696926233e-05, (4, 7): 2.9359254882058386e-06, (5, 6): 4.5669952038757497e-07, (6, 5): 5.074439115417505e-08, (7, 4): 4.027332631283742e-09, (8, 3): 2.2374070173798472e-10, (9, 2): 8.286692656962395e-12, (1, 10): 7.134298936340169e-05, (10, 1): 1.841487257102758e-13, (11, 0): 1.86008813848764e-15, (0, 12): 4.226415927827759e-05, (2, 10): 3.443746311563347e-05, (3, 9): 1.2754615968753153e-05, (4, 8): 3.1886539921882874e-06, (5, 7): 5.668718208334756e-07, (6, 6): 7.348338418211695e

{(0, 0): 4.978814792505536e-11, (0, 1): 0.025150360195919333, (1, 0): 0.1648398020747578, (0, 10): 9.977788006726225e-08, (2, 8): 1.916924685987083e-05, (3, 7): 4.977041286994973e-05, (4, 6): 0.00010830028811126658, (5, 5): 0.00021386899793863256, (6, 4): 0.0003645573957992809, (7, 3): 0.0004908431096215705, (8, 2): 0.0004737646010432444, (9, 1): 0.0002868040557190021, (1, 9): 4.483775640878049e-06, (10, 0): 8.117241989433969e-05, (0, 11): 4.2242146290395805e-08, (2, 9): 9.303612874284516e-06, (3, 8): 2.4458134373753283e-05, (4, 7): 5.318312480528961e-05, (5, 6): 0.0001078434611355338, (6, 5): 0.00020035677393238303, (7, 4): 0.00031590585468805455, (8, 3): 0.0003891163434993416, (9, 2): 0.00034274077383880365, (1, 10): 2.0907557938255463e-06, (10, 1): 0.00018976660678686222, (11, 0): 4.93270066847845e-05, (0, 12): 1.8527829517924202e-08, (2, 10): 4.646908016955649e-06, (3, 9): 1.2415317330701123e-05, (4, 8): 2.6839623379373354e-05, (5, 7): 5.452464119420256e-05, (6, 6): 0.0001060170387

{(0, 0): 3.3207540144705253e-11, (0, 1): 0.03069736256497428, (1, 0): 0.22351752590580037, (0, 10): 8.474993834805025e-09, (2, 8): 1.5475434046633629e-06, (3, 7): 3.670427432590849e-06, (4, 6): 1.057684190959021e-05, (5, 5): 3.820235285453157e-05, (6, 4): 0.00013069714056838225, (7, 3): 0.000347255611035374, (8, 2): 0.0006388626096689032, (9, 1): 0.0007157561475910553, (1, 9): 4.7482911339402637e-07, (10, 0): 0.0003666931757186821, (0, 11): 3.0802861672818692e-09, (2, 9): 6.271051507723815e-07, (3, 8): 1.3717658999040783e-06, (4, 7): 3.316022157184238e-06, (5, 6): 1.0974435183230088e-05, (6, 5): 4.0460168528611844e-05, (7, 4): 0.0001288919291927546, (8, 3): 0.0003105476635218416, (9, 2): 0.0005171968909674975, (1, 10): 1.9015918551979036e-07, (10, 1): 0.0005271910652704137, (11, 0): 0.000247311820733182, (0, 12): 1.1624705233258706e-09, (2, 10): 2.6492494340426686e-07, (3, 9): 5.567853772897709e-07, (4, 8): 1.1572785676189507e-06, (5, 7): 3.268006511881938e-06, (6, 6): 1.19650661737130

{(0, 0): 2.214865449791104e-11, (0, 1): 0.032496631523593084, (1, 0): 0.2443040604247498, (0, 10): 1.537071699500054e-09, (2, 8): 2.6031007784053625e-07, (3, 7): 6.781934087792297e-07, (4, 6): 3.0750160050398153e-06, (5, 5): 1.7134330913822598e-05, (6, 4): 7.927457973071939e-05, (7, 3): 0.00026599378818156365, (8, 2): 0.0005993880766617727, (9, 1): 0.0008100108268949672, (1, 9): 9.526706137629658e-08, (10, 0): 0.0004961361583996941, (0, 11): 4.980785896072939e-10, (2, 9): 9.092557322896717e-08, (3, 8): 1.9420380795540591e-07, (4, 7): 6.827141411387373e-07, (5, 6): 3.7702351605631613e-06, (6, 5): 1.998697966578013e-05, (7, 4): 8.251560744058354e-05, (8, 3): 0.0002461922634745608, (9, 2): 0.0004972201241342425, (1, 10): 3.394515123913964e-08, (10, 1): 0.0006077942731623661, (11, 0): 0.0003395926002199137, (0, 12): 1.6786506971922786e-10, (2, 10): 3.3628589076550855e-08, (3, 9): 6.3689824962211e-08, (4, 8): 1.6797449408947578e-07, (5, 7): 8.242402233702039e-07, (6, 6): 4.724948789868791e-

{(0, 0): 1.4772631238337445e-11, (0, 1): 0.03302420190890535, (1, 0): 0.2523187733292364, (0, 10): 4.2600050569071386e-10, (2, 8): 7.917848231708114e-08, (3, 7): 3.1354213280884123e-07, (4, 6): 2.1193793261468244e-06, (5, 5): 1.3866060801461738e-05, (6, 4): 6.933845025026999e-05, (7, 3): 0.0002462919090827773, (8, 2): 0.0005838675761286686, (9, 1): 0.0008280461730983256, (1, 9): 2.7389100479693115e-08, (10, 0): 0.0005315991643255512, (0, 11): 1.2230194374132379e-10, (2, 9): 2.3006537742512407e-08, (3, 8): 6.850054496104591e-08, (4, 7): 4.05625119796062e-07, (5, 6): 2.8581638027474064e-06, (6, 5): 1.6694956215579583e-05, (7, 4): 7.323457545369073e-05, (8, 3): 0.0002299784963816512, (9, 2): 0.0004873202276227825, (1, 10): 8.63283450049107e-09, (10, 1): 0.0006241209174371747, (11, 0): 0.0003650768839548596, (0, 12): 3.672486756291997e-11, (2, 10): 7.277024442284833e-09, (3, 9): 1.7124824459529277e-08, (4, 8): 8.067850990226304e-08, (5, 7): 5.739562144821798e-07, (6, 6): 3.760079869588588e

{(0, 0): 9.852999187200451e-12, (0, 1): 0.03336365621510288, (1, 0): 0.25627258353900934, (0, 10): 1.5062391855403523e-10, (2, 8): 3.7791638678449815e-08, (3, 7): 2.3637894629277005e-07, (4, 6): 1.9151973808115598e-06, (5, 5): 1.3118320122320654e-05, (6, 4): 6.691054114197049e-05, (7, 3): 0.00024117936656283881, (8, 2): 0.0005794316689100677, (9, 1): 0.0008324173112959233, (1, 9): 9.830030970518528e-09, (10, 0): 0.0005412256814734285, (0, 11): 3.757400220864778e-11, (2, 9): 8.879229183497735e-09, (3, 8): 4.425481351207891e-08, (4, 7): 3.5141435657852003e-07, (5, 6): 2.661817437510282e-06, (6, 5): 1.5923460984961954e-05, (7, 4): 7.091958719453398e-05, (8, 3): 0.00022569175239866386, (9, 2): 0.0004843765766359362, (1, 10): 2.688218381048425e-09, (10, 1): 0.0006281804858032227, (11, 0): 0.0003720487822820104, (0, 12): 9.899674503833886e-12, (2, 10): 2.307227938271413e-09, (3, 9): 8.995561282321667e-09, (4, 8): 6.529782420230911e-08, (5, 7): 5.240481648506677e-07, (6, 6): 3.543717879234163

{(0, 0): 6.571719444868834e-12, (0, 1): 0.033665561988166094, (1, 0): 0.2589194772066117, (0, 10): 6.607068440940356e-11, (2, 8): 2.550594062193656e-08, (3, 7): 2.1457413386566244e-07, (4, 6): 1.8604473662437122e-06, (5, 5): 1.2922645374436529e-05, (6, 4): 6.627622576508883e-05, (7, 3): 0.00023983581822290326, (8, 2): 0.0005782540728687225, (9, 1): 0.000833573023346917, (1, 9): 4.3589346853663836e-09, (10, 0): 0.0005438036634782182, (0, 11): 1.4069909986409527e-11, (2, 9): 5.073060877849735e-09, (3, 8): 3.792121272828828e-08, (4, 7): 3.374995361865174e-07, (5, 6): 2.6114041135241297e-06, (6, 5): 1.5723753377862318e-05, (7, 4): 7.031465030125075e-05, (8, 3): 0.00022455940478441274, (9, 2): 0.0004835822734797671, (1, 10): 1.016612768636155e-09, (10, 1): 0.0006292434255668857, (11, 0): 0.0003739116202947035, (0, 12): 3.188939105136348e-12, (2, 10): 1.0973920555457908e-09, (3, 9): 7.060088971092903e-09, (4, 8): 6.159916034013151e-08, (5, 7): 5.115884548434613e-07, (6, 6): 3.488303233839704

# Plot Reduced 

In [ ]:
m_to_cluster, cluster_to_m, m_to_weight = get_cluster_maps(clustering)
prints(cluster_to_m)

for timepoint in list(range(101)):
    m_to_value_baseline = m_to_value_t[timepoint]
    dynamics_reduced   #101 rows × 2652 columns
    dyn_o =  dynamics_reduced.to_dict(orient='records')
    dyn_line = dyn_o[timepoint]
    m_to_value = dict()
    for cluster, value in dyn_line.items():
        if 'S' in cluster:
            continue
        cluster = cluster.replace("('I', '", "").replace("')","")
        mlist = cluster_to_m[cluster]
        for m in mlist:  
            m_to_value[m] = value * m_to_weight[m]  #ORIGINAL
            m_to_value[m] = np.abs(m_to_value_baseline[m] - m_to_value[m])  # For Diff plot
    prints(m_to_value) 

    real_time = str(timepoint/101.0*3) + '00000000000000000000'
    real_time = real_time[:6]
    plot_dyn(m_to_value, 'DIFFReduced 73 Inf   t='+real_time, timepoint/101.0*3)